# Data Cleaning and Visualization

There are 23 columns and 145460 rows in this time-series dataset. The goal is to predict the `RainTomorrow` variable.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# helper function to get quick row info
def getDfRowInfo(dataframe):
    print('DF Row Info:')
    print('\tTotal number of rows:', len(df.index))
    print('\tPercentage of original number of rows:', len(df.index) / 145460)
    print('\tTotal number of rows with the RainTomorrow = true:', len([x for x in df['RainTomorrow'] if x == 'Yes']))
    print('\tPercentage of dataset that has RainTomorrow = true:', len([x for x in df['RainTomorrow'] if x == 'Yes']) / len(df.index))
    print('*********************************************************************************************')

# -----------------------
# read in dataset with date column parsed
df = pd.read_csv('weatherAUS.csv',
    parse_dates=['Date'],
    index_col='Date')

# preview data
getDfRowInfo(df)

# print out the number of non-NA values per column
print('Total number of non-NA values per column, out of', len(df), 'total columns:')
print(df.count(0))


DF Row Info:
	Total number of rows: 145460
	Percentage of original number of rows: 1.0
	Total number of rows with the RainTomorrow = true: 31877
	Percentage of dataset that has RainTomorrow = true: 0.21914615701911178
*********************************************************************************************
Total number of non-NA values per column, out of 145460 total columns:
Location         145460
MinTemp          143975
MaxTemp          144199
Rainfall         142199
Evaporation       82670
Sunshine          75625
WindGustDir      135134
WindGustSpeed    135197
WindDir9am       134894
WindDir3pm       141232
WindSpeed9am     143693
WindSpeed3pm     142398
Humidity9am      142806
Humidity3pm      140953
Pressure9am      130395
Pressure3pm      130432
Cloud9am          89572
Cloud3pm          86102
Temp9am          143693
Temp3pm          141851
RainToday        142199
RainTomorrow     142193
dtype: int64


In [2]:
# NA value management
# Only use rows of df that do not have NA in Rain Tomorrow, the label to predict
df = df[df['RainTomorrow'].notna()]

getDfRowInfo(df)

# replace numerical columns with mean and categorical columns with mode
for column in ['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 
               'Sunshine', 'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 
               'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 
               'Cloud9am', 'Cloud3pm', 'Temp9am', 'Temp3pm']:
    df[column] = df[column].fillna(df[column].mean())

for column in ['WindGustDir', 'WindDir9am', 'WindDir3pm']:
    df[column] = df[column].fillna(df[column].mode())
    
# ------------------------
# transform binary string to binary 0 or 1 values
df['RainToday'] = df['RainToday'].transform(lambda x: 0 if x == 'No' else 1)
df['RainTomorrow'] = df['RainTomorrow'].transform(lambda x: 0 if x == 'No' else 1)

# -------------------------
df.head()

DF Row Info:
	Total number of rows: 142193
	Percentage of original number of rows: 0.9775402172418535
	Total number of rows with the RainTomorrow = true: 31877
	Percentage of dataset that has RainTomorrow = true: 0.22418121848473554
*********************************************************************************************


,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
Date,,,,,,,,,,,,,,,,,,,,,
2008-12-01,Albury,13.4,22.9,0.6,5.469824,7.624853,W,44.0,W,WNW,...,71.0,22.0,1007.7,1007.1,8.000000,4.503167,16.9,21.8,0,0
2008-12-02,Albury,7.4,25.1,0.0,5.469824,7.624853,WNW,44.0,NNW,WSW,...,44.0,25.0,1010.6,1007.8,4.437189,4.503167,17.2,24.3,0,0
2008-12-03,Albury,12.9,25.7,0.0,5.469824,7.624853,WSW,46.0,W,WSW,...,38.0,30.0,1007.6,1008.7,4.437189,2.000000,21.0,23.2,0,0
2008-12-04,Albury,9.2,28.0,0.0,5.469824,7.624853,NE,24.0,SE,E,...,45.0,16.0,1017.6,1012.8,4.437189,4.503167,18.1,26.5,0,0
2008-12-05,Albury,17.5,32.3,1.0,5.469824,7.624853,W,41.0,ENE,NW,...,82.0,33.0,1010.8,1006.0,7.000000,8.000000,17.8,29.7,0,0


In [3]:
# useful syntax to slice data based on time:
# df["1/1/2010":"1/2/2010"].head() # gets all data in the given date range


In [4]:
import matplotlib.pyplot as plt


In [6]:
import plotly.express as px

# group the data by location and calculate the average min and max temperatures
grouped = df.groupby('Location').agg({'MinTemp': 'mean', 'MaxTemp': 'mean'}).reset_index()

# create the interactive bar chart using plotly
fig = px.bar(grouped, x='Location', y=['MinTemp', 'MaxTemp'], 
             title='Average Min and Max Temperatures by Location',
             barmode='group',
             labels={'variable': 'Temperature', 'value': 'Temperature (°C)'})

# show the interactive chart
fig.show()
print(fig)


Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'Temperature=MinTemp<br>Location=%{x}<br>Temperature (°C)=%{y}<extra></extra>',
              'legendgroup': 'MinTemp',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': 'MinTemp',
              'offsetgroup': 'MinTemp',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['Adelaide', 'Albany', 'Albury', 'AliceSprings', 'BadgerysCreek',
                          'Ballarat', 'Bendigo', 'Brisbane', 'Cairns', 'Canberra', 'Cobar',
                          'CoffsHarbour', 'Dartmoor', 'Darwin', 'GoldCoast', 'Hobart',
                          'Katherine', 'Launceston', 'Melbourne', 'MelbourneAirport', 'Mildura',
                          'Moree', 'MountGambier', 'MountGinini', 'Newcastle', 'Nhil',
                          'NorahHead', 'NorfolkIsland', 'Nuriootpa

In [7]:
# calculate the average rainfall per year
avg_rainfall = df.groupby(df.index.year)['Rainfall'].mean()

# create the line chart using plotly express
fig = px.bar(avg_rainfall, x=avg_rainfall.index, y=avg_rainfall.values,
              title='Average Rainfall per Year',
              labels={'x': 'Year', 'y': 'Rainfall (mm)'})

# show the interactive chart
fig.show()
print(fig)


Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'Date=%{x}<br>Rainfall (mm)=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': '',
              'offsetgroup': '',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': array([2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]),
              'xaxis': 'x',
              'y': array([3.21967213, 2.30683418, 2.15985826, 2.7020156 , 2.80969498, 2.40481229,
                          2.26681964, 1.9608014 , 2.13721759, 2.37302637, 2.4695662 ]),
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Average Rainfall per Year'},
               'xaxis': {'anchor': 'y', 'domain': 

In [8]:
# read in dataset with date column parsed
df = pd.read_csv('weatherAUS.csv',
    parse_dates=['Date'],
    index_col='Date')

# calculate average evaporation and average sunshine by month
monthly_avg = df.groupby(pd.Grouper(freq='M')).mean()
monthly_avg = monthly_avg[['Evaporation', 'Sunshine']].reset_index()

# create the scatter plot using plotly express
fig = px.scatter(monthly_avg, x='Evaporation', y='Sunshine', 
                 title='Relationship Between Average Evaporation and Average Sunshine',
                 labels={'Evaporation': 'Average Evaporation (mm)', 'Sunshine': 'Average Sunshine (hours)'})

fig.show()
print(fig)


Figure({
    'data': [{'hovertemplate': 'Average Evaporation (mm)=%{x}<br>Average Sunshine (hours)=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': array([5.9       , 5.81935484, 7.94193548, 5.65614035, 5.67741935, 3.995     ,
                          2.50967742, 1.95666667, 2.665     , 3.40932203, 4.95391304, 6.07631579,
                          6.27623318, 7.14728507, 9.18924051, 7.85913853, 6.29968717, 4.4613808 ,
                          2.98169456, 2.22087432, 2.68859275, 3.82750809, 5.0563818 , 5.85553145,
                          8.03162584, 8.39989189, 8.89426681, 7.42660333, 5.90064171, 3.92735327,
                          2.82975295, 2.26883978, 2.21961207, 3.10268757, 3.77411504, 5.07440273,
                          6.028

In [9]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# group data by year and month
rain_data = df.groupby([df.index.year, df.index.month])['Rainfall'].sum()

# create a new dataframe with columns for year, month, and rainfall
rain_df = pd.DataFrame({'Year': [y for y, _ in rain_data.index],
                        'Month': [m for _, m in rain_data.index],
                        'Rainfall': rain_data.values})

# pivot the dataframe to create columns for each year
rain_pivot = rain_df.pivot(index='Month', columns='Year', values='Rainfall')

# create subplot
fig = make_subplots(rows=1, cols=1)

# add traces for each year
for year in rain_pivot.columns:
    fig.add_trace(go.Scatter(x=rain_pivot.index, y=rain_pivot[year],
                             mode='lines', name=str(year)),
                  row=1, col=1)

# set layout
fig.update_layout(title='Cumulative Annual Rainfall by Year',
                  xaxis_title='Month',
                  yaxis_title='Cumulative Annual Rainfall')

# show the plot
fig.show()
print(fig)


Figure({
    'data': [{'mode': 'lines',
              'name': '2007',
              'type': 'scatter',
              'x': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]),
              'xaxis': 'x',
              'y': array([  nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
                           95.4, 101. ]),
              'yaxis': 'y'},
             {'mode': 'lines',
              'name': '2008',
              'type': 'scatter',
              'x': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]),
              'xaxis': 'x',
              'y': array([  43.8,  323. ,   93.4,  164. ,   15.8,  149. ,  569.4,  273.6,  331.4,
                           352.6,  741. , 2092. ]),
              'yaxis': 'y'},
             {'mode': 'lines',
              'name': '2009',
              'type': 'scatter',
              'x': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]),
              'xaxis': 'x',
              'y': array([2951.1, 4888.2, 1964.3, 3

In [10]:
# transform WindDir and Location columns into encoded labels
la = LabelEncoder()
l = []
for i in df.columns:
    if df.dtypes[i]=='O':
        l.append(i)
        df[i] = la.fit_transform(df[i])
print('Transformed columns to encoded labels: ', l)

# -------------------------
# preview data
print(df.describe())
df.head()

Transformed columns to encoded labels:  ['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday', 'RainTomorrow']
            Location        MinTemp        MaxTemp       Rainfall  \
count  145460.000000  143975.000000  144199.000000  142199.000000   
mean       23.793524      12.194034      23.221348       2.360918   
std        14.228687       6.398495       7.119049       8.478060   
min         0.000000      -8.500000      -4.800000       0.000000   
25%        11.000000       7.600000      17.900000       0.000000   
50%        24.000000      12.000000      22.600000       0.000000   
75%        36.000000      16.900000      28.200000       0.800000   
max        48.000000      33.900000      48.100000     371.000000   

        Evaporation      Sunshine    WindGustDir  WindGustSpeed  \
count  82670.000000  75625.000000  145460.000000  135197.000000   
mean       5.468232      7.611178       8.306565      40.035230   
std        4.193704      3.785483       4.971722    

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
Date,,,,,,,,,,,,,,,,,,,,,
2008-12-01,2,13.4,22.9,0.6,NaN,NaN,13,44.0,13,14,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,0,0
2008-12-02,2,7.4,25.1,0.0,NaN,NaN,14,44.0,6,15,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,0,0
2008-12-03,2,12.9,25.7,0.0,NaN,NaN,15,46.0,13,15,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,0,0
2008-12-04,2,9.2,28.0,0.0,NaN,NaN,4,24.0,9,0,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,0,0
2008-12-05,2,17.5,32.3,1.0,NaN,NaN,13,41.0,1,7,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,0,0


In [11]:
# write cleaned dataset to new file

df.to_csv('cleanedWeatherAUS.csv')